In [ ]:
import pandas as pd

# Load the initial contact database
db_pomoci = pd.read_csv('/mnt/data/df_scraped.csv')

# Load the scraped data from Mapy.cz
maps_results = pd.read_excel('/mnt/data/maps_results.xlsx')

# Load the combined scraped data with phone numbers as strings
combined_df = pd.read_csv('/mnt/data/combined_output.csv', dtype={'Contact': str})

# Ensure phone numbers are formatted correctly
def format_phone_number(phone):
    if pd.isna(phone):
        return phone
    phone = phone.split('.')[0]  # Remove the .0 if present
    if not phone.startswith('+'):
        phone = '+' + phone  # Add the country code if missing
    return phone

# Apply the formatting to the 'Contact' column for phone numbers
combined_df.loc[combined_df['Contact_type'] == 'Phone', 'Contact'] = combined_df.loc[combined_df['Contact_type'] == 'Phone', 'Contact'].apply(format_phone_number)

# Combine emails and phone numbers from maps_results
maps_contacts = pd.concat([
    maps_results[['Emails_scraped']].rename(columns={'Emails_scraped': 'Contact'}),
    maps_results[['formated_number']].rename(columns={'formated_number': 'Contact'}).applymap(format_phone_number)
]).dropna().drop_duplicates()

# Extract contacts from combined_df
scraped_contacts = combined_df[['Contact']].dropna().drop_duplicates()

# Function to check if a contact exists in the scraped data
def check_contact(contact, contacts_df):
    return contact in contacts_df['Contact'].values

# Check and flag contacts in db_pomoci
db_pomoci['Matched'] = db_pomoci.apply(
    lambda row: 'matched' if check_contact(row['Emails'], maps_contacts) or 
                           check_contact(row['Phone'], maps_contacts) or 
                           check_contact(row['Emails'], scraped_contacts) or 
                           check_contact(row['Phone'], scraped_contacts) 
                else 'unmatched', axis=1)

# Function to find a new contact
def find_new_contact(row):
    if row['Matched'] == 'unmatched':
        web = row['Base Website']
        # Check for new contacts in maps_contacts and scraped_contacts
        maps_contact = maps_contacts[maps_contacts['Contact'].str.contains(web, na=False)]
        scraped_contact = scraped_contacts[scraped_contacts['Contact'].str.contains(web, na=False)]
        
        if not maps_contact.empty and not scraped_contact.empty:
            # Both sources have the contact
            new_contact = maps_contact.iloc[0]['Contact']
            return pd.Series([new_contact, 'new_contact_both_match'])
        elif not maps_contact.empty:
            # Only maps_contacts has the contact
            new_contact = maps_contact.iloc[0]['Contact']
            return pd.Series([new_contact, 'new_contact_scraped_maps'])
        elif not scraped_contact.empty:
            # Only scraped_contacts has the contact
            new_contact = scraped_contact.iloc[0]['Contact']
            return pd.Series([new_contact, 'new_contact_scraped_combined'])
    return pd.Series([None, None])

# Apply the find_new_contact function to the DataFrame
db_pomoci[['New Contact', 'New Matched']] = db_pomoci.apply(find_new_contact, axis=1)

# Update the contact information and flag accordingly
db_pomoci['Matched'] = db_pomoci.apply(
    lambda row: row['New Matched'] if pd.notna(row['New Matched']) else row['Matched'], axis=1)
db_pomoci['Emails'] = db_pomoci.apply(
    lambda row: row['New Contact'] if pd.isna(row['Emails']) and pd.notna(row['New Contact']) else row['Emails'], axis=1)
db_pomoci['Phone'] = db_pomoci.apply(
    lambda row: row['New Contact'] if pd.isna(row['Phone']) and pd.notna(row['New Contact']) else row['Phone'], axis=1)

# Drop the helper columns
db_pomoci.drop(columns=['New Contact', 'New Matched'], inplace=True)

# Save the resulting DataFrame to a new CSV file
db_pomoci.to_csv('/mnt/data/db_pomoci_flagged_with_new_contacts.csv', index=False)

# Print the resulting DataFrame
print(db_pomoci)
